# Data Pipeline

## Part 1: Webscraping

### Install packages

In [15]:
import html5lib
import bs4
import requests
from bs4 import BeautifulSoup

### Access contents

In [12]:
headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9"}
URLS = ["https://www.vox.com/donald-trump/401174/trump-fired-generals", "https://www.foxnews.com/world/trump-says-minerals-deal-has-been-pretty-much-negotiated-zelenskyy-meeting-slated-friday"]
for url in URLS:
    r = requests.get(url, headers=headers)
    print(r.content)

b'<!DOCTYPE html><html lang="en-US"><meta name="bingbot" content="nocache"/><head><meta charSet="utf-8"/><meta name="twitter:card" content="summary_large_image"/><meta name="twitter:site" content="@voxdotcom"/><meta property="fb:app_id" content="549923288395304"/><meta property="og:site_name" content="Vox"/><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"/><meta name="apple-mobile-web-app-title" content="Vox"/><link rel="alternate" type="application/rss+xml" title="Vox" href="/rss/index.xml"/><title>Trump, Hegseth purge key military leaders. Here\xe2\x80\x99s what\xe2\x80\x99s at stake. | Vox</title><meta name="robots" content="index,follow,max-image-preview:large"/><meta name="description" content="How the Friday night massacre could change America\xe2\x80\x99s armed forces."/><meta property="og:title" content="What Trump\xe2\x80\x99s military purge was really about"/><meta property="og:description" content="How the Friday night massacre could chan

### Parse contents

In [23]:
soup = BeautifulSoup(r.content, 'html5lib')
print(soup.prettify())

<!DOCTYPE html>
<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22en%22%7D%7D" data-n-head-ssr="" lang="en">
 <head>
  <title>
   Volodymyr Zelenskyy, Donald Trump come to agreement on minerals deal | Fox News
  </title>
  <meta content="IE=edge,chrome=1" data-n-head="ssr" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" data-hid="content-type" data-n-head="ssr" http-equiv="content-type"/>
  <meta content="on" data-hid="x-dns-prefetch-control" data-n-head="ssr" http-equiv="x-dns-prefetch-control"/>
  <meta charset="utf-8" data-n-head="ssr"/>
  <meta content="width=device-width, minimum-scale=1.0, initial-scale=1.0" data-hid="viewport" data-n-head="ssr" name="viewport"/>
  <meta content="//static.foxnews.com/static/orion/styles/img/fox-news/favicons/mstile-70x70.png" data-n-head="ssr" name="msapplication-square70x70logo"/>
  <meta content="//static.foxnews.com/static/orion/styles/img/fox-news/favicons/mstile-150x150.png" data-n-head="ssr" name="msapplication-squa

In [49]:
headline_text=[]
table = soup.find('script')
for row in table.find_all('script', attrs = {'headline':'articleBody'}):
    headline = row.find('headline').text.strip()
    text = row.find('articleBody').text.strip()
    title_text.append({'headline': headline, 'text':text})

In [47]:
headline_text

[]